In [1]:
from datetime import datetime
horizon = 6
train_end = datetime(year=2023, month=5, day=31)

In [9]:
import json
 
# Data to be written
scenario_data = {
    #start_date
     'Start_date':              str(train_end),
    # ССВ
     'SSV':                  [0.2 for h in range(horizon)],
    # ФОР
     'FOR':                  [0.2 for h in range(horizon)],
    # Трансфертные ставки
     'VTB_ftp_rate_[90d]':   [5 for h in range(horizon)],
     'VTB_ftp_rate_[180d]':  [5.2 for h in range(horizon)],
     'VTB_ftp_rate_[365d]':  [5.7 for h in range(horizon)],
     'VTB_ftp_rate_[548d]':  [5 for h in range(horizon)],
     'VTB_ftp_rate_[730d]':  [5.2 for h in range(horizon)],
     'VTB_ftp_rate_[1095d]': [5.4 for h in range(horizon)],
    
    # Маржа бизнеса по срочностям
     'margin_[90d]':         [0.7 for h in range(horizon)],
     'margin_[180d]':        [0.7 for h in range(horizon)],
     'margin_[365d]':        [0.7 for h in range(horizon)],
     'margin_[548d]':        [0.7 for h in range(horizon)],
     'margin_[730d]':        [0.7 for h in range(horizon)],
     'margin_[1095d]':       [0.7 for h in range(horizon)],
    
    # Спред Привилегия - Массовый (на сколько в среднем ставки по сегменту Привилегия больше чем ставки по массовому сегменту)
     'priv_spread':          [0.5 for h in range(horizon)],
    # Спред ВИП - Массовый (на сколько в среднем ставки по сегменту ВИП больше чем ставки по массовому сегменту)
     'vip_spread':           [0.8 for h in range(horizon)],
    
    # Ниже три спреда по разным типам опциональности по отношению к безопциональным вкладам (Подразумевается, что они, как правило, отрицательные)
    # r - возможности пополнения, s - возможность расходных операций
    
    # На сколько ставка по расходным вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r0s1_spread':          [-0.2 for h in range(horizon)],
    
    # На сколько ставка по пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s0_spread':          [-0.2 for h in range(horizon)],
    
    # На сколько ставка по расходно-пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s1_spread':          [-0.2 for h in range(horizon)],
    
    # Ставка по лучшему предложению сбера
     'SBER_max_rate':        [5.5 for h in range(horizon)],
    
    # Базовая ставка по НС
     'SA_rate':              [5 for h in range(horizon)]
}
 

scenario_df_user = pd.DataFrame(scenario_data)
# Serializing json
json_object = json.dumps(scenario_data, indent=4)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [2]:
import pandas as pd

In [18]:
df=pd.read_excel("dynbalance/examples/agg_res.xlsx")

In [19]:
df['start_balance_agg_report_dt']=df['balance']-df['balance_gain']

In [20]:
df_CA=pd.read_excel("dynbalance/examples/agg_res.xlsx",sheet_name="CurrentAccounts")
df_SA=pd.read_excel("dynbalance/examples/agg_res.xlsx",sheet_name="SavingAccounts")

In [21]:
df_CA['segment']='mass'
df1=df_CA.copy()
df1['segment']='vip'
df_CA=pd.concat([df_CA,df1])
df1['segment']='priv'
df_CA=pd.concat([df_CA,df1])
df_CA.reset_index(drop=True,inplace=True)

In [34]:
results_dict={}
for report_dt in df['report_dt'].unique():
    segment_dict={}
    for segment in  df['segment'].unique():
        results_dict[str(report_dt)[0:10]]=[]
        dep_dict=[]
        sa_dict={}
        ca_dict={}
        sub_df=df[(df['report_dt']==report_dt)&(df['segment']==segment)]
        sub_df_SA=df_SA[(df_SA['report_dt']==report_dt)&(df_SA['segment']==segment)]
        sub_df_CA=df_CA[(df_CA['report_dt']==report_dt)&(df_CA['segment']==segment)]
        for i in range(len(sub_df)):
            dep_dict.append({'replenishable_flg': str(df['replenishable_flg'][i]),
                                        'subtraction_flg': str(df['subtraction_flg'][i]),
                                        'target_maturity_days': str(df['target_maturity_days'][i]),
                                        'start_balance': str(df['start_balance_agg_report_dt'][i]),
                                        'balance_gain': str(df['balance_gain'][i]),
                                        'balance': str(df['balance'][i]),
                                        'newbusiness': str(df['newbusiness'][i]),
                                        'contract_close': str(df['contract_close'][i]),
                                        'early_withdrawal': str(df['early_withdrawal'][i]),
                                        'operations': str(df['operations'][i]),
                                        'interests'	: str(df['interests'][i]),
                                        'renewal'	: str(df['renewal'][i])
                
            })
        sa_dict.update({"general":float(sub_df_SA[['general']].values),
                        "kopilka":float(sub_df_SA[['kopilka']].values),
                        "safe":float(sub_df_SA[['safe']].values)
                       })
        ca_dict.update({"current_accounts_balance_rur":float(sub_df_CA[['current_accounts_balance_rur']].values) })
        inter_dict={"Deposits":dep_dict}
        inter_dict.update({"SavingAccounts":sa_dict})
        inter_dict.update({"CurrentAccounts":ca_dict})
        segment_dict.update({segment:inter_dict})
        
    results_dict.update({str(report_dt)[0:10]:segment_dict})


In [40]:
# Serializing json
json_object = json.dumps(results_dict, indent=4)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

23/06/29 20:02:04 WARN security.UserGroupInformation: Exception encountered while running the renewal command for vtb70174694@REGION.VTB.RU. (TGT end time:1688065324000, renewalFailures: 0, renewalFailuresTotal: 1)
ExitCodeException exitCode=1: kinit: Bad format in credentials cache (filename: /home/vtb70174694/.krb5cc_1000650088) while storing credentials

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1009)
	at org.apache.hadoop.util.Shell.run(Shell.java:902)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1227)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1321)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1303)
	at org.apache.hadoop.security.UserGroupInformation$TicketCacheRenewalRunnable.relogin(UserGroupInformation.java:1077)
	at org.apache.hadoop.security.UserGroupInformation$AutoRenewalForUserCredsRunnable.run(UserGroupInformation.java:988)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.